# wordSim評価
# wordAnalogy評価

# -----------初期設定-----------

In [1]:
# -*- coding: utf-8 -*-
# python3
#
import argparse
import gzip
import math
import numpy
import re
import sys
import numpy as np
from copy import deepcopy
import codecs

In [2]:
isNumber = re.compile(r'\d+.*')
def norm_word(word):
    if isNumber.search(word.lower()):
        return '---num---'
    elif re.sub(r'\W+', '', word) == '':
        return '---punc---'
    else:
        return word.lower()

In [3]:
"""Read all the word vectors and normalize them"""
def read_word_vecs(filename):
    wordVectors = {}
    # ファイル読み込み
    if filename.endswith('.gz'): 
        fileObject = gzip.open(filename, 'r')
    else: 
        fileObject = codecs.open(filename, "r", "utf-8", 'ignore')
        
    for line in fileObject:
        # line = line.strip().lower()
        line = line.strip()
        word = line.split()[0]
        wordVectors[word] = numpy.zeros(len(line.split())-1, dtype=float)
        for index, vecVal in enumerate(line.split()[1:]):
            wordVectors[word][index] = float(vecVal)
        """normalize weight vector"""
        wordVectors[word] /= math.sqrt((wordVectors[word]**2).sum() + 1e-6)

    sys.stderr.write("Vectors read from: "+filename+" \n")
    return wordVectors

In [4]:
"""Write word vectors to file"""
def print_word_vecs(wordVectors, outFileName):
    sys.stderr.write('\nWriting down the vectors in '+outFileName+'\n')
    outFile = open(outFileName, 'w')  
    for word, values in wordVectors.items():
        outFile.write(word+' ')
        for val in wordVectors[word]:
            outFile.write('%.4f' %(val)+' ')
        outFile.write('\n')      
    outFile.close()

In [5]:
"""Read the PPDB.etc word relations as a dictionary"""
def read_lexicon(filename):
    lexicon = {}
    fileObject = open(filename, 'r')
    for line in fileObject:
        words = line.lower().strip().split()
        lexicon[norm_word(words[0])] = [norm_word(word) for word in words[1:]]
    return lexicon

In [6]:
"""Retrofit word vectors to a lexicon"""
def retrofit(wordVecs, lexicon, numIters):
    # Input word vecs
    newWordVecs = deepcopy(wordVecs)
    # Input word vecsの単語リスト
    wvVocab = set(newWordVecs.keys())
    # wvVocabとlexiconの共通単語
    loopVocab = wvVocab.intersection(set(lexicon.keys()))

    for _ in range(numIters): #10回程度
        # loop through every node also in ontology (else just use data estimate)
        for word in loopVocab:
            # lexicon wordの近傍単語とwvVocabの共通単語とその個数
            wordNeighbours = set(lexicon[word]).intersection(wvVocab)
            numNeighbours = len(wordNeighbours)
            # no neighbours -> pass - use data estimate
            if numNeighbours == 0:
                continue
            """分散表現の更新手続き"""
            # the weight of the data estimate if the number of neighbours
            newVec = numNeighbours * wordVecs[word]
            # loop over neighbours and add to new vector (currently with weight 1)
            for ppWord in wordNeighbours:
                newVec += newWordVecs[ppWord]
            newWordVecs[word] = newVec/(2*numNeighbours)
    return newWordVecs

In [7]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1) # v1のノルム
    n2 = np.linalg.norm(v2) # v2のノルム
    return np.dot(v1, v2) / (n1*n2) # 内積 / 

## ・パス指定

In [8]:
# オリジナルの分散表現
input_arg = './word2vec/vectors.model'
# wordnetの辞書
lexicon_arg = './result.txt'
# retrofittingしたnewvec
output_arg = './out_vec.txt'

In [9]:
# Num iterations
numiter_arg = 10

## ・初期vecとnewvecとwordsim辞書のread

In [10]:
numIter = int(numiter_arg)

In [11]:
outFileName = output_arg

In [12]:
lexicon = read_lexicon(lexicon_arg)

In [13]:
wordVecs = read_word_vecs(input_arg)

Vectors read from: ./word2vec/vectors.model 


In [14]:
new_vec = read_word_vecs(output_arg)

Vectors read from: ./out_vec.txt 


# -----------WordSim評価-----------

In [15]:
# word2vec
vecs_wordVecs = wordVecs

In [16]:
# retrofitting
vecs_new_vec = new_vec

In [36]:
def checkSim(v1, v2):
    if v1 not in lexicon: # 注目単語がwordnetに含まれない場合，
        print("v1(={})はwordnetのkeyに存在しません".format(v1))
        wordNeighbours_1 = set()
    else:
        # 注目単語の同義語リストとword2vecのkeyリストと重複する単語リスト（更新対象か？）
        wordNeighbours_1 = set(lexicon[v1]).intersection(set(wordVecs.keys()))
        print('v1(={})における更新対象のneighbour数 : {}'.format(v1, len(wordNeighbours_1)))
        
    # neighboursの中で注目単語と類似度が低いもの
    ave_1_word2vec = np.zeros_like(wordVecs[v1])  #neighboursの平均ベクトル
    ave_1_retrofit = np.zeros_like(wordVecs[v1])
    for neighbour in wordNeighbours_1:
        print('v1とneighbour(={}) : {} -> {}'.format(neighbour, 
                                                                   similarity(wordVecs[neighbour], wordVecs[v1]), 
                                                                   similarity(vecs_new_vec[neighbour], vecs_new_vec[v1])))
        ave_1_word2vec += wordVecs[neighbour]
        ave_1_retrofit += vecs_new_vec[neighbour]
    print('v1とneighboursの平均ベクトル : {} -> {}'.format(similarity(ave_1_word2vec/len(wordNeighbours_1), wordVecs[v1]), 
                                                              similarity(ave_1_retrofit/len(wordNeighbours_1), vecs_new_vec[v1])))
    print(" ")
    
    if v2 not in lexicon:
        print("v2(={})はwordnetのkeyに存在しません".format(v2))
        wordNeighbours_2 = set()
    else:
        wordNeighbours_2 = set(lexicon[v2]).intersection(set(wordVecs.keys()))
        print('v2(={})における更新対象のneighbour数 : {}'.format(v2, len(wordNeighbours_2))) # 更新対象数
    
    # neighboursの中で注目単語と類似度が低いもの
    ave_2_word2vec = np.zeros_like(wordVecs[v2])  #neighboursの平均ベクトル
    ave_2_retrofit = np.zeros_like(wordVecs[v2])
    for neighbour in wordNeighbours_2:
        print('v2とneighbour(={}) : {} -> {}'.format(neighbour, 
                                                                   similarity(wordVecs[neighbour], wordVecs[v2]), 
                                                                   similarity(vecs_new_vec[neighbour], vecs_new_vec[v2])))
        ave_2_word2vec += wordVecs[neighbour]
        ave_2_retrofit += vecs_new_vec[neighbour]
    print('v2とneighboursの平均ベクトル : {} -> {}'.format(similarity(ave_2_word2vec/len(wordNeighbours_2), wordVecs[v2]), 
                                                              similarity(ave_2_retrofit/len(wordNeighbours_2), vecs_new_vec[v2])))
    print(" ")
    
    # v1とv2における更新対象のneighbourに重複があるか
    print('更新対象における重複 : {}'.format(set(wordNeighbours_1).intersection(set(wordNeighbours_2))))
    print('更新対象における重複数 : {}'.format(len(set(wordNeighbours_1).intersection(set(wordNeighbours_2)))))
    print(" ")

    """v1とv2のword2vecとretrofittingにおける類似度"""
    try:
        print("word2vec : {}".format(similarity(vecs_wordVecs[v1], vecs_wordVecs[v2])))
    except:
        print('error')

    try:
        print("retrofitting : {}".format(similarity(vecs_new_vec[v1], vecs_new_vec[v2])))
    except:
        print('error')

    print(" ")
    """同一単語におけるword2vecとretrofittingの類似度"""
    try:
        print("v1 : {}".format(similarity(vecs_wordVecs[v1], vecs_new_vec[v1])))
    except:
        print('error')

    try:
        print("v2 : {}".format(similarity(vecs_wordVecs[v2], vecs_new_vec[v2])))
    except:
        print('error')

## ・WordSimのテスト

In [34]:
v1 = 'フットボール'
v2 = 'サッカー'
# checkSim(v1, v2)

In [19]:
v1 = 'ドル'
v2 = '円'
# checkSim(v1, v2)

In [60]:
v1 = '間'
v2 = '時間'
checkSim(v1, v2)

v1(=間)における更新対象のneighbour数 : 84
v1とneighbour(=繋がり) : 0.21342898562479795 -> 0.4704273621131348
v1とneighbour(=関連) : 0.20028334657127156 -> 0.4598616756235437
v1とneighbour(=頼り) : 0.1420511445578004 -> 0.4294354413661628
v1とneighbour(=中休) : 0.3623476196967441 -> 0.5768493144794318
v1とneighbour(=タイム) : 0.09461590080008835 -> 0.4162744891592635
v1とneighbour(=切目) : 0.24241745652714772 -> 0.49463456340601514
v1とneighbour(=間断) : 0.2136323528169702 -> 0.49988761717862207
v1とneighbour(=コネクション) : 0.22698561769472408 -> 0.4847429430037322
v1とneighbour(=切れ目) : 0.26547957637923114 -> 0.5188652249481165
v1とneighbour(=小休) : 0.20640387701025137 -> 0.4913633054696056
v1とneighbour(=ひと息) : 0.08731866656599313 -> 0.42964353061655974
v1とneighbour(=間隔) : 0.44589792479299173 -> 0.6554575511332801
v1とneighbour(=間合い) : 0.2868401360441464 -> 0.558988211216215
v1とneighbour(=ブレーク) : 0.17344648384321548 -> 0.4545684039655927
v1とneighbour(=中入り) : 0.13441843588037428 -> 0.44517292033876044
v1とneighbour(=合間) : 0.317231

In [63]:
v1 = '足'
v2 = '交通手段'
checkSim(v1, v2)

v1(=足)における更新対象のneighbour数 : 31
v1とneighbour(=足下) : 0.3857479176881243 -> 0.5905864793587807
v1とneighbour(=交通手段) : 0.300802704374794 -> 0.5489118627304612
v1とneighbour(=鳥目) : 0.31390175669188636 -> 0.5710542761849712
v1とneighbour(=通運) : 0.12809331353851025 -> 0.4275384392097293
v1とneighbour(=金子) : 0.22149034045881025 -> 0.5099070388389261
v1とneighbour(=足元) : 0.4482701929937417 -> 0.6187163250924947
v1とneighbour(=脚) : 0.6138774297145551 -> 0.7486451002682474
v1とneighbour(=銀子) : 0.1711036874788435 -> 0.4809907643312406
v1とneighbour(=ゲル) : 0.13548733018638942 -> 0.46418036374360894
v1とneighbour(=お金) : 0.23939181169632903 -> 0.4914846415249338
v1とneighbour(=足もと) : 0.3372794715771023 -> 0.5565535196200979
v1とneighbour(=金銭) : 0.20123901305796307 -> 0.4742061774446219
v1とneighbour(=金円) : 0.25956176094111694 -> 0.5519071721176427
v1とneighbour(=交通網) : 0.26481051656296023 -> 0.5321410373231001
v1とneighbour(=丸物) : 0.15758124122715855 -> 0.4732261785604359
v1とneighbour(=足部) : 0.3856471457437549 -> 

In [92]:
v1 = '切る'
v2 = '混ぜる'
checkSim(v1, v2)

v1(=切る)における更新対象のneighbour数 : 79
v1とneighbour(=句切る) : 0.23680833603045093 -> 0.5213595549558694
v1とneighbour(=成し遂げる) : 0.23137618784745967 -> 0.45707871491784374
v1とneighbour(=別ける) : 0.14839166375869986 -> 0.408651889166032
v1とneighbour(=取止める) : 0.21293213883054 -> 0.5103299298384512
v1とneighbour(=区画) : 0.19555653183631092 -> 0.43826425421974946
v1とneighbour(=分かつ) : 0.1149997218182326 -> 0.418255539906376
v1とneighbour(=打切る) : 0.2285784087737064 -> 0.5206755618004563
v1とneighbour(=裁つ) : 0.3923758804955995 -> 0.6600817308410425
v1とneighbour(=刈り込む) : 0.27338561252757954 -> 0.5598015733500732
v1とneighbour(=裁断) : 0.3020140134765606 -> 0.5395004251986637
v1とneighbour(=仕上げる) : 0.29325470347201693 -> 0.5577451395927542
v1とneighbour(=中止) : 0.1157558400933728 -> 0.40984485713496704
v1とneighbour(=カット) : 0.3855937142429067 -> 0.5939917695046227
v1とneighbour(=取りやめる) : 0.15512417755773605 -> 0.44918518756583314
v1とneighbour(=剪定) : 0.2864007297760136 -> 0.5851313761094258
v1とneighbour(=差し止める) : 0.0997

In [101]:
v1 = '最後'
v2 = 'ゴール'
checkSim(v1, v2)

v1(=最後)における更新対象のneighbour数 : 97
v1とneighbour(=終局) : 0.2208766651605024 -> 0.5461299707841252
v1とneighbour(=止め) : 0.2589320321143105 -> 0.5669881867157595
v1とneighbour(=端っこ) : 0.22907876748674524 -> 0.4681471171593578
v1とneighbour(=先端) : 0.1605448481732543 -> 0.4162075380602424
v1とneighbour(=端末) : 0.1282980159962607 -> 0.426163519672441
v1とneighbour(=結び) : 0.194539608892796 -> 0.49793410601157884
v1とneighbour(=没) : 0.22996741674378662 -> 0.5131021198726348
v1とneighbour(=終止) : 0.25020420583081415 -> 0.5455042851545923
v1とneighbour(=落命) : 0.2295890469843856 -> 0.5095025215822989
v1とneighbour(=結尾) : 0.27909235928909853 -> 0.5776553028204098
v1とneighbour(=挙げ句の果て) : 0.15778797822380383 -> 0.4905386248606585
v1とneighbour(=死歿) : 0.3215910685347451 -> 0.5789926352054033
v1とneighbour(=不祝儀) : 0.2043014271009203 -> 0.5047314496421658
v1とneighbour(=最期) : 0.49267923633689586 -> 0.7355837857746595
v1とneighbour(=絶命) : 0.2712655098675786 -> 0.5480728159134739
v1とneighbour(=歿) : 0.23057067792925043 -> 0

In [115]:
v1 = '国'
v2 = '分野'
checkSim(v1, v2)

v1(=国)における更新対象のneighbour数 : 8
v1とneighbour(=分野) : 0.2803379969335132 -> 0.5379740306537499
v1とneighbour(=ステート) : 0.11621479878160752 -> 0.45178619158660116
v1とneighbour(=国家) : 0.6145990089996953 -> 0.8249719104913206
v1とneighbour(=世界) : 0.34595373863134665 -> 0.5803956246533929
v1とneighbour(=邦) : 0.35177829496825813 -> 0.7017259161601905
v1とneighbour(=聯邦) : 0.29987704556928546 -> 0.6625696550340545
v1とneighbour(=連邦) : 0.3197057387385614 -> 0.6099376846243704
v1とneighbour(=領域) : 0.23806846144136745 -> 0.5260305599655913
v1とneighboursの平均ベクトル : 0.5442038398834504 -> 0.814465147786048
 
v2(=分野)における更新対象のneighbour数 : 50
v2とneighbour(=分配) : 0.2554990107022538 -> 0.4962427922807238
v2とneighbour(=区画) : 0.2715708262189193 -> 0.524962509432658
v2とneighbour(=除法) : 0.18313617641648594 -> 0.4891150213010623
v2とneighbour(=節) : 0.18245728258739263 -> 0.4411031766791021
v2とneighbour(=域) : 0.2557525800487358 -> 0.5483048650908964
v2とneighbour(=畑) : 0.1980165239949193 -> 0.48497513523318025
v2とneighbour(

In [47]:
v1 = '甘い'
v2 = '寛大'
checkSim(v1, v2)

v1(=甘い)における更新対象のneighbour数 : 6
v1とneighbour(=緩やか) : 0.2415450710133865 -> 0.6585850594842375
v1とneighbour(=寛仁) : 0.16929136236203007 -> 0.57879291095691
v1とneighbour(=緩い) : 0.3472129470097726 -> 0.6794209905851345
v1とneighbour(=寛大) : 0.3171064549414952 -> 0.6499191017923927
v1とneighbour(=寛容) : 0.26343233800063826 -> 0.6278198393501258
v1とneighbour(=手ぬるい) : 0.3400710271855983 -> 0.6721585199652284
v1とneighboursの平均ベクトル : 0.4361472174522379 -> 0.781154711090697
 
v2(=寛大)における更新対象のneighbour数 : 41
v2とneighbour(=懇切) : 0.4126655574042469 -> 0.6829323015488185
v2とneighbour(=温かい) : 0.3558699775183507 -> 0.6437110756046661
v2とneighbour(=親身) : 0.39518350217246456 -> 0.6712992257214935
v2とneighbour(=容赦) : 0.45909483311236027 -> 0.6859781446035553
v2とneighbour(=暖か) : 0.29620875367099975 -> 0.6065537282121106
v2とneighbour(=でっかい) : 0.1593182639866135 -> 0.4430178257337308
v2とneighbour(=雅量) : 0.2103319811669914 -> 0.5740793600221223
v2とneighbour(=免罪符) : 0.2642540644549206 -> 0.7411855187049984
v2とneigh

In [37]:
v1 = '首'
v2 = '解雇'
checkSim(v1, v2)

v1(=首)における更新対象のneighbour数 : 26
v1とneighbour(=ヘッド) : 0.24642739616787185 -> 0.3834064549051069
v1とneighbour(=免職) : 0.1892702002683228 -> 0.5070718312111325
v1とneighbour(=頸部) : 0.4505123507331748 -> 0.6889985970126411
v1とneighbour(=頚) : 0.366792699959356 -> 0.6506117126706568
v1とneighbour(=罷免) : 0.20959893777745434 -> 0.5182192690193763
v1とneighbour(=免官) : 0.17965117617729778 -> 0.49464775280618123
v1とneighbour(=暇) : 0.2666812654209634 -> 0.4971757180074593
v1とneighbour(=ご免) : 0.14204212399357322 -> 0.4598029626345756
v1とneighbour(=頚部) : 0.3676779115420377 -> 0.6540707103982555
v1とneighbour(=解職) : 0.1312684605500554 -> 0.46552335306519044
v1とneighbour(=免) : 0.19956668039363393 -> 0.505980854861111
v1とneighbour(=解任) : 0.12804974618421514 -> 0.47080391908473695
v1とneighbour(=解雇) : 0.16909683470493422 -> 0.49536208845757723
v1とneighbour(=首切り) : 0.3281515923962968 -> 0.5831062374404989
v1とneighbour(=衿) : 0.26896706421372424 -> 0.5695506306426479
v1とneighbour(=御免) : 0.16202428996720836 -> 0.4

# -----------ある単語の類似する単語を挙げる-----------

In [44]:
def checkSim_by_word(vecs, word):
    # 閾値の設定
    negative = False # Falseなら似た単語を候補で上げる
    threshold = 0.5 # -1なら閾値固定
    border_positive = threshold if threshold > 0 else 0.8
    border_negative = threshold if threshold > 0 else 0.3

    # 候補数の設定
    max_candidates = 20
    candidates = {}
    
    # wordの設定確認
    if not word:
        raise Exception("word is missing")

    # wordがモデルにない場合，
    if word not in vecs:
        raise Exception("Sorry, this word is not registered in model.")

    # ベクトルの設定
    w_vec = vecs[word]

    # ナレッジグラフにあるかどうかの確認
    lexicon = read_lexicon(lexicon_arg)
    if word not in lexicon:
    #     raise Exception("not found error in dict")
        print("not found error in dict")
    
    for w in vecs:
        try:
            if w_vec.shape != vecs[w].shape:
                raise Exception("size not match")
            s = similarity(w_vec, vecs[w])
        except Exception as ex:
            print(w + " is not valid word.")
            continue

        if negative and s <= border_negative:
            candidates[w] = s
            if len(candidates) % 5 == 0:
                border_negative -= 0.05
        elif not negative and s >= border_positive:
            candidates[w] = s
            if len(candidates) % 5 == 0:
                border_positive += 0.05

        if len(candidates) > max_candidates:
            break
    
    # 類義語算出
    sorted_candidates = sorted(candidates, key=candidates.get, reverse=not negative)
    for c in sorted_candidates:
        print("{0}, {1}".format(c, candidates[c]))

In [45]:
word = '首'

In [46]:
# 初期vecの場合，
checkSim_by_word(vecs_wordVecs, word)
print(' ')
# newvecの場合，
checkSim_by_word(vecs_new_vec, word)

754069 is not valid word.
首, 1.0
首に, 0.7020332391035057
刎, 0.6211267347905213
続拾遺和歌集, 0.5661742831081139
新拾遺和歌集, 0.5642845831301884
続後拾遺和歌集, 0.5585148861206279
尾, 0.553421708581033
腰, 0.5351856466237928
背中, 0.5103645327180215
手足, 0.5028418379039111
 
754069 is not valid word.
首, 1.0
雁首, 0.7145410570579709
首に, 0.7080931894414839
頸, 0.705496105537352
頸部, 0.6889985970126411
腰, 0.6660742726043445
頚部, 0.6540707103982555
背中, 0.6395247544907925
斬る, 0.6324658384173607
胸, 0.6289739001145982
尾, 0.6179651471587841
腹, 0.6096465211436369
頭, 0.5883427801536355
頭部, 0.5761437111868011
脚, 0.5717697691129308
足, 0.5715720429533705
殺す, 0.5612531885843647
体, 0.5498965134414835
身, 0.5185022070429954
口, 0.5169285861383547
手, 0.5127556266930783


# -----------wordAnalogy評価-----------

In [26]:
def checkAnalogy(vecs, w_vec):  
    negative = False # Falseなら似た単語を候補で上げる
    threshold = 0.3 # -1なら閾値固定

    # 閾値の設定
    border_positive = threshold if threshold > 0 else 0.9
    border_negative = threshold if threshold > 0 else 0.2
    print('{} < thd < {}'.format(border_negative, border_positive))

    # 候補数の設定
    max_candidates = 20
    candidates = {}

    for w in vecs:
        try:
            if w_vec.shape != vecs[w].shape:
                raise Exception("size not match")
            s = similarity(w_vec, vecs[w])
        except Exception as ex:
            print(w + " is not valid word.")
            continue

        if negative and s <= border_negative:
            candidates[w] = s
            if len(candidates) % 5 == 0:
                border_negative -= 0.05
        elif not negative and s >= border_positive:
            candidates[w] = s
            if len(candidates) % 5 == 0:
                border_positive += 0.05

        if len(candidates) > max_candidates:
            break

    # 類義語算出
    sorted_candidates = sorted(candidates, key=candidates.get, reverse=not negative)
    for c in sorted_candidates:
        print("{0}, {1}".format(c, candidates[c]))

## ・「v1 + v2 - v3」と「v4」の類似度算出

In [27]:
v1 = '兄'
v2 = '姉'
v3 = '祖父'
v4 = '祖母'

In [28]:
if v1 not in lexicon:
    print("v1 not found error in dict")
if v2 not in lexicon:
    print("v2 not found error in dict")
if v3 not in lexicon:
    print("v3 not found error in dict")
if v4 not in lexicon:
    print("v4 not found error in dict")

try:
    print('word2vec : {}'.format(similarity(vecs_wordVecs[v1] + vecs_wordVecs[v2] - vecs_wordVecs[v3], vecs_wordVecs[v4])))
except:
    print('error')

try:
    print('retrofitting : {}'.format(similarity(vecs_new_vec[v1] + vecs_new_vec[v2] - vecs_new_vec[v3], vecs_new_vec[v4])))
except:
    print('error')

word2vec : 0.3855693677632116
retrofitting : 0.46470763443894203


## ・「v1 + v2 - v3」と近い単語を挙げる→「v4」が結果に出るか

In [29]:
# 初期vecの場合，
checkAnalogy(vecs_wordVecs, vecs_wordVecs[v1] + vecs_wordVecs[v2] - vecs_wordVecs[v3])
print(' ')
# newvecの場合，
checkAnalogy(vecs_new_vec, vecs_new_vec[v1] + vecs_new_vec[v2] - vecs_new_vec[v3])

0.3 < thd < 0.3
754069 is not valid word.
姉, 0.8433279735101462
兄, 0.7622779357580127
妹, 0.7553375375315672
弟, 0.7011033258786433
双子, 0.5541898022518956
夫, 0.5241162973051148
年上, 0.5228285283574702
次女, 0.49053484292919564
兄弟, 0.4801388417438813
叔父, 0.48004084731816965
甥, 0.4646477248521072
母, 0.4618720699656536
長女, 0.4522498603989746
娘。, 0.4484254297338895
彼女, 0.41874909255787995
妻, 0.4180556386599047
息子, 0.38730153006548584
父, 0.3860297212008132
結婚, 0.3574390676533354
男, 0.35215392646403093
2人, 0.32434570907744076
 
0.3 < thd < 0.3
754069 is not valid word.
弟, 0.735540018182597
彼女, 0.5708677571388876
母, 0.543018985376124
息子, 0.5335641681270236
妻, 0.5269046411495635
娘。, 0.5115487458496082
女性, 0.4747866974855413
子供, 0.46754522886759947
彼, 0.464979913115947
父, 0.4579658210443133
結婚, 0.4532385554739422
男性, 0.4523917286989586
子, 0.4158272591171027
2人, 0.41108535260447876
自分, 0.3953707483145449
思う, 0.376416952702512
出演, 0.36485803889961343
メンバー, 0.35210435795316825
後, 0.3356467345493145
活動,